In [ ]:
!pip install pytube
!pip install pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
!pip install presets

  Preparing metadata (setup.py) ... done
  Created wheel for presets: filename=presets-0.1.3-py3-none-any.whl size=3913 sha256=0666efb40f16171443797bc53eba1d8cd5f40c80bae8c6de880f87c3a73e3cac
  Stored in directory: /root/.cache/pip/wheels/43/e0/50/8659018b68466d540eda58d53ef6fe6e75c6b1457ce3162a0a
Successfully built presets


In [ ]:
!pip install openai
!pip install cohere
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.9 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1
ERROR: pip's dependency resolver

In [ ]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 34.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import librosa as _librosa
import numpy as np
import math
from presets import Preset
import time

In [ ]:
import os
from pytube import YouTube
from pydub import AudioSegment
import time
from IPython.display import clear_output
import pandas as pd

In [ ]:
import speech_recognition as sr
import os

In [ ]:
def get_urls():
  with open(os.getcwd() + "/drive/MyDrive/data/audio/data_urls/to_get.txt") as f:
    urls = f.read().splitlines()

  return urls

def record_url_process_progress(finished_url):

  remaining_urls = ""

  processed_urls = ""

  with open(os.getcwd() + "/drive/MyDrive/data/audio/data_urls/to_get.txt", "r") as f:
    for line in f.read().splitlines():
      if line != finished_url and line != "\n":
        remaining_urls += line + "\n"

  with open(os.getcwd() + "/drive/MyDrive/data/audio/data_urls/to_get.txt", "w") as f:
    f.write(remaining_urls)

  with open(os.getcwd() + "/drive/MyDrive/data/audio/data_urls/processed.txt", "r") as f:
      for line in f.read().splitlines():
        if line != "\n":
          processed_urls += line + "\n"
      processed_urls += f"{finished_url} data_index={audio_count}\n"

  with open(os.getcwd() + "/drive/MyDrive/data/audio/data_urls/processed.txt", "w") as f:
    f.write(processed_urls)


def label_segment(segment_to_label, bitrate):
  recognizer = sr.Recognizer()

  segment_to_label.export(os.getcwd() + f"/drive/MyDrive/data/audio/processed_audio/temp.wav", format="wav", bitrate=bitrate)

  with sr.AudioFile(os.getcwd() + f"/drive/MyDrive/data/audio/processed_audio/temp.wav") as source:
    audio = recognizer.record(source)

  try:
      text = recognizer.recognize_whisper(audio, language="turkish", model="large")
      print("Transcription: " + text)

  except sr.UnknownValueError:
      print("Google Speech Recognition could not understand the audio")

  except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition; {0}".format(e))

  return text


def pre_process_and_label_audio(url):
  length_of_segment = 60000
  audio_count = 31
  video = YouTube(url)
  video.title = f"{audio_count}_audio"
  audio_title = video.title
  audio_stream = video.streams.filter(only_audio=True, file_extension="mp4").first()
  bitrate = audio_stream.abr[0:-3]

  audio_stream.download(output_path=os.getcwd() + "/drive/MyDrive/data/audio/raw_audio")


  audio_to_convert : AudioSegment = AudioSegment.from_file(os.getcwd() + f"/drive/MyDrive/data/audio/raw_audio/{audio_title}.mp4")

  audio_to_convert = audio_to_convert.set_frame_rate(16000)

  num_of_segments = len(audio_to_convert) // length_of_segment + 1

  audio_segments = []

  segment_label = []

  if len(audio_to_convert) % length_of_segment != 0:
    silent_record = AudioSegment.silent(length_of_segment - len(audio_to_convert) % length_of_segment)
    audio_to_convert += silent_record

  for i in range(num_of_segments):
    audio_segments.append(audio_to_convert[0 + i * length_of_segment : (i + 1) * length_of_segment])
    segment_labels.append(label_segment(audio_segments[i], bitrate))

  return audio_segments, segment_labels

In [ ]:
pre_process_and_label_audio("https://www.youtube.com/watch?v=kyRXWArpl8Y")

  2%|▉                                    | 70.4M/2.88G [00:11<07:50, 6.40MiB/s]


KeyboardInterrupt: 

In [ ]:
print(label_audio(get_and_pre_process_audio("https://www.youtube.com/watch?v=kyRXWArpl8Y")[0]))

60000


FileNotFoundError: [Errno 2] No such file or directory: '3'

In [ ]:
while True:
  audio_count = count_audio_number("/drive/MyDrive/data/audio/raw_audio")
  try:
    url_to_process = get_urls()[0]

  except IndexError:
    break;

  audio_title = get_and_pre_process_audio(url_to_process)
  extract_features(audio_title, batch_length_in_miliseconds)
  record_url_process_progress(url_to_process)